In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as ppt


In [5]:
bank = pd.read_csv('D:\\Downloads\\bank-additional\\bank-additional.csv', sep=';')

In [6]:
bank.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


In [74]:
dfMarital = pd.get_dummies(bank['marital'])
dfJob = pd.get_dummies(bank['job'])
dfEducation = pd.get_dummies(bank['education'])

bank1hotted = bank.merge(dfMarital, left_index=True, right_index=True)
bank1hotted = bank1hotted.merge(dfJob, left_index=True, right_index=True)
bank1hotted = bank1hotted.merge(dfEducation, left_index=True, right_index=True)


In [75]:
# Drop unnecessary columns: check the columns
bank1hotted.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y', 'divorced', 'married',
       'single', 'unknown_x', 'admin.', 'blue-collar', 'entrepreneur',
       'housemaid', 'management', 'retired', 'self-employed', 'services',
       'student', 'technician', 'unemployed', 'unknown_y', 'basic.4y',
       'basic.6y', 'basic.9y', 'high.school', 'illiterate',
       'professional.course', 'university.degree', 'unknown'],
      dtype='object')

In [80]:
# Drop unnecessary columns: drop them for real
# pick on the relevant columns, drop the rest
bank1hotted['y'].replace(('yes', 'no'), (True, False), inplace=True)

bank1hottedFeatures = bank1hotted.drop(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month',
                                        'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate',
                                        'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'unknown_x',
                                       'unknown_y', 'unknown'], axis=1)


In [81]:
bank1hottedFeatures['y'].head(15)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
Name: y, dtype: bool

In [82]:
# separate the training-testing data

from sklearn.model_selection import train_test_split

train, test = train_test_split(bank1hottedFeatures, test_size=0.2)

train.shape, test.shape

((3295, 23), (824, 23))

The possible values of marital are - single, married, divorced, unknown. Unknown will hurt the model more when present than when taken out. Unknowns will not have a behaviour of its own. They will be one of the other 3.

In [89]:
from sklearn.naive_bayes import GaussianNB

nbClass = GaussianNB()
trainWithoutLabel = train.drop('y', axis=1)
nbModel = nbClass.fit(trainWithoutLabel, train['y'])

In [90]:
testWithoutLabel = test.drop('y', axis=1)

predY = nbModel.predict(testWithoutLabel)


In [131]:
test.at[774,'y']

False

In [132]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(classification_report(test.y, predY))
print(confusion_matrix(test.y, predY))

              precision    recall  f1-score   support

       False       0.91      0.36      0.51       719
        True       0.15      0.75      0.24       105

   micro avg       0.41      0.41      0.41       824
   macro avg       0.53      0.55      0.38       824
weighted avg       0.81      0.41      0.48       824

[[257 462]
 [ 26  79]]
